<a href="https://colab.research.google.com/github/kareem1925/Classical-and-quantum-regression-analysis-for-the-optoelectronic-performance-of-NTCDA/blob/master/QNN_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing required packages into colab and refreshing the environment


In [ ]:
!pip install pennylane==0.10.0 strawberryfields==0.14.0 pennylane-sf==0.9.0 -q
import os
os.kill(os.getpid(), 9)

## Attention
If you are using colab, you should clone the repo below so that you can import some helper functions.

In [2]:
!git clone https://github.com/kareem1925/Classical-and-quantum-regression-analysis-for-the-optoelectronic-performance-of-NTCDA

Cloning into 'Classical-and-quantum-regression-analysis-for-the-optoelectronic-performance-of-NTCDA'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 81 (delta 38), reused 26 (delta 8), pack-reused 0
Unpacking objects: 100% (81/81), done.


In [6]:
cd Classical-and-quantum-regression-analysis-for-the-optoelectronic-performance-of-NTCDA/

/content/Classical-and-quantum-regression-analysis-for-the-optoelectronic-performance-of-NTCDA


In [8]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer,NesterovMomentumOptimizer,AdagradOptimizer,RMSPropOptimizer
from sklearn.utils import shuffle
import strawberryfields as sf
from strawberryfields.ops import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from utils import testing,accuracy,next_batch,data_preparation

In [ ]:
"""

A quantum device tht has 18 dimensions and the hbar is set to 2

"""
dev = qml.device('strawberryfields.fock', wires=1, cutoff_dim=18,hbar=2)

## The quantum layer that would be repeated 8 times.

In [10]:
def ansatz(v):
    qml.Rotation(v[0], wires=0)
    qml.Squeezing(v[1], 0, wires=0)
    qml.Rotation(v[2], wires=0)
    qml.Displacement(v[3], 0, wires=0)
    qml.Kerr(v[4], wires=0)

## The quantum node

In [11]:
@qml.qnode(dev)
def qnn(var, x=None):
    
    # Encoding the volt and intensity into quantum states
    qml.DisplacedSqueezedState(x[0], 0,x[1],0, wires=0)

    for v in (var):
        ansatz(v)

    return qml.expval(qml.X(0))

### Computing the trace of the system

In [12]:
@qml.qnode(dev)
def trace(var, x=None):
    
    qml.DisplacedSqueezedState(x[0],0,x[1],0,wires=0)
    
    for v in var:
    
        ansatz(v)
    
    return qml.expval(qml.Identity([0]))

### The cost function

In [13]:
def cost(var, features, labels):

    out = 0

    for i in range(len(features)):
        
        preds = qnn(var,x=features[i])
    
        out = out + (preds-labels[i])**2
    
    traces = [trace(var, x=x) for x in features]
    
    traces = (1-abs(np.array(traces))).mean()

    return (out/len(features))+0.01*traces

## Preparing the dataset for computation as mentioned in the paper

In [14]:
X_train, X_test, y_train, y_test = data_preparation()

## The results that we achieved from training

In [15]:
best_weights = np.load('./assets/qnn_weights_per_epoch.npy')[363]

test_pred = testing(best_weights,X_test)

train_pred = testing(best_weights,X_train)

testing_results = accuracy(y_test/1000,test_pred/1000)

training_results = accuracy(y_train/1000,train_pred/1000)

print('testing results: \n','MSE: ',testing_results[0],' | ',
      'MAE: ',testing_results[1],' | ', 'R^2: ',testing_results[2])

print('training results: \n','MSE: ',training_results[0],' | ',
      'MAE: ',training_results[1], ' | ', 'R^2: ',training_results[2])


testing results: 
 MSE:  9.695218817688523e-11  |  MAE:  7.676295551862258e-06  |  R^2:  0.9985132119674823
training results: 
 MSE:  1.2655411646766855e-10  |  MAE:  8.801097994493057e-06  |  R^2:  0.9985142035733218


## Training loop

In [ ]:
weight_epoch=[]
test = []
weights = []    
mse_v = []

np.random.seed(1220)
var_init = np.random.normal(0,0.001,(8,5))
var = var_init
opt = AdamOptimizer(0.01, beta1=0.9, beta2=0.999)
opt.reset()
old_acc = - np.inf
update_var = var

for it in range(10000):
    
    now = time.time()
    
    batch_test = np.random.choice(X_test.shape[0],X_test.shape[0],replace=False)
    
    X_test1 = X_test[batch_test]
    y_test1 = y_test[batch_test]
  
    batch = np.random.choice(X_train.shape[0],X_train.shape[0],replace=False)
    
    feats_train_11 =X_train[batch]
    
    Y_train1 = y_train[batch]

    for id,(batchX, batchY) in enumerate(next_batch(feats_train_11, Y_train1,32)):
        
        var = opt.step(lambda v: cost(v, batchX, batchY), var)
        weights.append(var)
        

    t=trace(var,x=X_train[5]) # arbitrary point to check for the sanity of the quntum state
    print("Trace: {:0.5f}".format(t))   
    
    predictions = testing(var,X_test1)
    ww = accuracy(y_test1, predictions)
    acc_test = ww[2]
    acc_m = ww[0]
    weight_epoch.append(var)
    np.save('weights_per_epoch',np.array(weight_epoch))

    test.append(acc_test)
    np.save('testing',test)
    mse_v.append(acc_m)
    
    if acc_test > old_acc:
        update_var = var
        old_acc = acc_test
        print("  ")
        print('model improved')
        print("  ")
    
    print("Iter:{:5d} | mse: {:0.7e} | mae: {:0.7e} | R2: {:0.5f}".format(it + 1,ww[0],ww[1],ww[2]),'time:', int((time.time()-now)))